In [18]:
import numpy as np
import random
import cv2

im = np.zeros((224,224,3), np.uint8) 
noise = cv2.randu(im,(0),(255))   
image = cv2.imwrite('noise_image1.png',noise) # Only for grayscale image

In [ ]:
import os
import time
import torch
import argparse
import numpy as np
import json

from utils.EEGDataset import EEGDataset
from sklearn.metrics.pairwise import cosine_similarity
from utils.DinoModel import DinoModel, dino_args
from utils.Utilities import initlogger
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

In [ ]:
def initDinoModel(model_to_load, FLAGS, checkpoint_key="teacher"):
    dino_args.pretrained_weights = model_to_load
    dino_args.output_dir = FLAGS.log_dir
    dino_args.checkpoint_key = checkpoint_key
    dino_args.use_cuda = torch.cuda.is_available()
    dinov1_model = DinoModel(dino_args)
    dinov1_model.eval()
    return dinov1_model
dinov1_model = initDinoModel(model_to_load=FLAGS.custom_model_weights,FLAGS=FLAGS,checkpoint_key="teacher")

In [ ]:
test_dataset = EEGDataset(subset=FLAGS.query_gallery,
                              eeg_signals_path=EEG_DATASET_PATH,
                              eeg_splits_path=EEG_DATASET_SPLIT,
                              subject=FLAGS.query_subject,
                              exclude_subjects=[],
                              preprocessin_fn=dinov1_model.dinov1_transform)    
dataset = EEGDataset(subset=FLAGS.search_gallery,
                        eeg_signals_path=EEG_DATASET_PATH,
                        eeg_splits_path=EEG_DATASET_SPLIT, 
                        subject=SUBJECT,
                        exclude_subjects=[],
                        preprocessin_fn=dinov1_model.dinov1_transform)

dataset.transformEEGDataDino(
                            model=dinov1_model,
                            preprocessor=dinov1_model.dinov1_transform,
                            pass_eeg=False,
                            device=device,
                            min_time=20,
                            max_time=490,
                            keep_features_flat=False)
